## In this Notebook, We will convert the model artifact to a TF lite model 
### Which is a nessecry step to deploy the model on the PSoC 6 board 

## Configure Defaults

**MODIFY** the following constants for your specific use case.

In [1]:
!pip install -r utils/keras_rewrite/requirements.txt

You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All the other words will be used to train an "unknown" label and silent
# audio data with no spoken words will be used to train a "silence" label.
WANTED_WORDS = "yes,no"

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=12000,3000 and LEARNING_RATE=0.001,0.0001
# will run 12,000 training loops in total, with a rate of 0.001 for the first
# 8,000, and 0.0001 for the final 3,000.
TRAINING_STEPS = "12000,3000"
LEARNING_RATE = "0.001,0.0001"

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
TOTAL_STEPS = str(sum(map(lambda string: int(string), TRAINING_STEPS.split(","))))

# Print the configuration to confirm it
print("Training these words: %s" % WANTED_WORDS)
print("Training steps in each stage: %s" % TRAINING_STEPS)
print("Learning rate in each stage: %s" % LEARNING_RATE)
print("Total number of training steps: %s" % TOTAL_STEPS)

Training these words: yes,no
Training steps in each stage: 12000,3000
Learning rate in each stage: 0.001,0.0001
Total number of training steps: 15000


**DO NOT MODIFY** the following constants as they include filepaths used in this notebook and data that is shared during training and inference.

In [3]:
# Calculate the percentage of 'silence' and 'unknown' training samples required
# to ensure that we have equal number of samples for each label.
number_of_labels = WANTED_WORDS.count(',') + 1
number_of_total_labels = number_of_labels + 2 # for 'silence' and 'unknown' label
equal_percentage_of_training_samples = int(100.0/(number_of_total_labels))
SILENT_PERCENTAGE = equal_percentage_of_training_samples
UNKNOWN_PERCENTAGE = equal_percentage_of_training_samples

# Constants which are shared during training and inference
PREPROCESS = 'micro'
WINDOW_STRIDE = 20
MODEL_ARCHITECTURE = 'tiny_conv' # Other options include: single_fc, conv,
                      # low_latency_conv, low_latency_svdf, tiny_embedding_conv

# Constants used during training only
VERBOSITY = 1
EVAL_STEP_INTERVAL = '1000'
SAVE_STEP_INTERVAL = '1000'

# Constants for training directories and filepaths
DATASET_DIR =  'dataset/'
LOGS_DIR = 'logs/'
TRAIN_DIR = 'train/' # for training checkpoints and other files.
NO_MLFLOW= True

# Constants for inference directories and filepaths
import os
MODELS_DIR = 'models'
if not os.path.exists(MODELS_DIR):
  os.mkdir(MODELS_DIR)

MODEL_TF = MODELS_DIR + '/model_2.pb'
MODEL_TFLITE = MODELS_DIR + '/tflite/model_final.tflite'
FLOAT_MODEL_TFLITE = MODELS_DIR + '/tflite/float_model_final.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + '/tflite/model_final.cc'
SAVED_MODEL = MODELS_DIR + '/saved_model'

QUANT_INPUT_MIN = 0.0
QUANT_INPUT_MAX = 26.0
QUANT_INPUT_RANGE = QUANT_INPUT_MAX - QUANT_INPUT_MIN

## Setup Environment

Install Dependencies

In [4]:
import tensorflow as tf
from tensorflow import keras

**DELETE** any old data from previous runs


In [5]:
# !rm -rf {LOGS_DIR} {TRAIN_DIR} 

Clone the TensorFlow Github Repository, which contains the relevant code required to run this tutorial.

Load TensorBoard to visualize the accuracy and loss as training proceeds.


## Training

The following script downloads the dataset and begin training.

Uncomment to start training. leave it commented if you would like to use pretarined model.

for training you need to change the name of the laod-weights argument by the latest model created by the training script 

In [20]:
# !python ../discard/keras_rewrite/train.py \
# --data_dir={DATASET_DIR} \
# --wanted_words={WANTED_WORDS} \
# --silence_percentage={SILENT_PERCENTAGE} \
# --unknown_percentage={UNKNOWN_PERCENTAGE} \
# --preprocess={PREPROCESS} \
# --window_stride={WINDOW_STRIDE} \
# --model_architecture={MODEL_ARCHITECTURE} \
# --how_many_training_steps={TRAINING_STEPS} \
# --learning_rate={LEARNING_RATE} \
# --train_dir={TRAIN_DIR} \
# --summaries_dir={LOGS_DIR} \
# --verbosity={VERBOSITY} \
# --eval_step_interval={EVAL_STEP_INTERVAL} \
# --save_step_interval={SAVE_STEP_INTERVAL} \
# --no_mlflow

2022-05-17 21:40:38.677969: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-05-17 21:40:38.678097: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-05-17 21:40:38.708032: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
File already exists at dataset/speech_commands_v0.02.tar.gz
Gathering .wav files from dataset/ : 100%|█| 105835/105835 [00:00<00:00, 118413.
Reading data index...
Validation partition: 1339 entries
Testing partition: 1374 entries
Training partition: 10598 entries
2022-05-17 21:40:58.241344: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them

## Generate a TensorFlow Model for Inference

Combine relevant training results (graph, weights, etc) into a single file for inference. This process is known as freezing a model and the resulting model is known as a frozen model/graph, as it cannot be further re-trained after this process.

In [9]:
# !rm -rf {SAVED_MODEL}
!python ./utils/keras_rewrite/freeze.py \
--wanted_words={WANTED_WORDS} \
--model_architecture={MODEL_ARCHITECTURE} \
--load_weights=./train/run_local/ckpt/weights-10-0.8843.h5 \
--output={SAVED_MODEL} \
--input_shape='(1960)'

2022-06-20 15:59:09.095663: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-06-20 15:59:09.095782: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-06-20 15:59:09.122774: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-06-20 15:59:10.835352: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-20 15:59:10.839897: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for b

## Generate a TensorFlow Lite Model

Convert the frozen graph into two TensorFlow Lite model. The first one is without quantization "Float_Model". The second one is fully quantized for use with embedded devices. 

The following cell will also print the model size, which will be under 20 kilobytes.

## Float TFlite Model

In [37]:
# write the float tflite model
!python ./utils/keras_rewrite/generate_tflite.py \
--input={SAVED_MODEL}  \
--output={FLOAT_MODEL_TFLITE} \
--cfile 

2022-05-17 22:46:02.416449: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-05-17 22:46:02.416589: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-05-17 22:46:02.525551: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-05-17 22:46:04.292657: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-17 22:46:04.293856: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for b

## Quantized TFlite Model

In [38]:
# write the quantized model
!python ./utils/keras_rewrite/generate_tflite.py \
--input={SAVED_MODEL} \
--output={MODEL_TFLITE} \
--int_only \
--data_dir={DATASET_DIR} \
--preprocess={PREPROCESS} \
--window_stride={WINDOW_STRIDE} \
--cfile 

2022-05-17 22:47:54.553288: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-05-17 22:47:54.553417: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-05-17 22:47:54.589940: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2022-05-17 22:47:56.125178: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-17 22:47:56.127890: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for b